In [1]:
# General imports
import numpy as np
import torch

# DeepMoD stuff
from deepymod import DeepMoD
from deepymod.model.func_approx import NN
from deepymod.model.library import Library1D
from deepymod.model.constraint import LeastSquares
from deepymod.model.sparse_estimators import Threshold
from deepymod.training.sparsity_scheduler import TrainTestPeriodic, Periodic, TrainTest

from deepymod.data import Dataset
from deepymod.data.burgers import BurgersDelta

from deepymod.utils.logger import Logger
from deepymod.training.convergence import Convergence

from deepymod.analysis import load_tensorboard

if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'
device='cpu'
# Settings for reproducibility
np.random.seed(42)
torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# Training function

In [2]:
def train_bayes(model: DeepMoD,
          data: torch.Tensor,
          target: torch.Tensor,
          optimizer,
          extra_params, 
          sparsity_scheduler,
          split = 0.8,
          exp_ID: str = None,
          log_dir: str = None,
          max_iterations: int = 10000,
          write_iterations: int = 25,
          **convergence_kwargs) -> None:
    """Trains the DeepMoD model. This function automatically splits the data set in a train and test set. 

    Args:
        model (DeepMoD):  A DeepMoD object.
        data (torch.Tensor):  Tensor of shape (n_samples x (n_spatial + 1)) containing the coordinates, first column should be the time coordinate.
        target (torch.Tensor): Tensor of shape (n_samples x n_features) containing the target data.
        optimizer ([type]):  Pytorch optimizer.
        sparsity_scheduler ([type]):  Decides when to update the sparsity mask.
        split (float, optional):  Fraction of the train set, by default 0.8.
        exp_ID (str, optional): Unique ID to identify tensorboard file. Not used if log_dir is given, see pytorch documentation.
        log_dir (str, optional): Directory where tensorboard file is written, by default None.
        max_iterations (int, optional): [description]. Max number of epochs , by default 10000.
        write_iterations (int, optional): [description]. Sets how often data is written to tensorboard and checks train loss , by default 25.
    """
    logger = Logger(exp_ID, log_dir)
    sparsity_scheduler.path = logger.log_dir # write checkpoint to same folder as tb output.
    
    t, a, l = extra_params
    
    # Splitting data, assumes data is already randomized
    n_train = int(split * data.shape[0])
    n_test = data.shape[0] - n_train
    data_train, data_test = torch.split(data, [n_train, n_test], dim=0)
    target_train, target_test = torch.split(target, [n_train, n_test], dim=0)
    
    M = 12
    N = data_train.shape[0]
    
    # Training
    convergence = Convergence(**convergence_kwargs)
    for iteration in torch.arange(0, max_iterations):
        # ================== Training Model ============================
        prediction, time_derivs, thetas = model(data_train)
        
        tau_ = torch.exp(t)
        alpha_ = torch.min(torch.exp(a), torch.tensor(1e8, dtype=torch.float32))
        lambda_ = torch.min(torch.exp(l), torch.tensor(2e4, dtype=torch.float32))
                            
        y = time_derivs[0]
        X = thetas[0]
        
        p_MSE = N / 2 * (tau_ * torch.mean((prediction - target_train)**2, dim=0) - t + np.log(2*np.pi))
        
        A = lambda_ * torch.eye(M) + alpha_ * X.T @ X
        mn = alpha_ * torch.inverse(A) @ X.T @ y
        E = alpha_ * torch.sum((y - X @ mn)**2) + lambda_ * mn.T @ mn
        p_reg = 1/2 * (E + torch.sum(torch.log(torch.diag(A))) - (M * l + N * a) - N * np.log(2*np.pi))
        
                         

        MSE = torch.mean((prediction - target_train)**2, dim=0)  # loss per output
        Reg = torch.stack([torch.mean((dt - theta @ coeff_vector)**2)
                           for dt, theta, coeff_vector in zip(time_derivs, thetas, model.constraint_coeffs(scaled=False, sparse=True))])
        loss = torch.sum(p_MSE + p_reg)

        # Optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if iteration % write_iterations == 0:
            # ================== Validation costs ================
            with torch.no_grad():
                prediction_test = model.func_approx(data_test)[0]
                MSE_test = torch.mean((prediction_test - target_test)**2, dim=0)  # loss per output
         
            # ====================== Logging =======================
            _ = model.sparse_estimator(thetas, time_derivs) # calculating estimator coeffs but not setting mask
            logger(iteration, 
                   loss, MSE, Reg,
                   model.constraint_coeffs(sparse=True, scaled=True), 
                   model.constraint_coeffs(sparse=True, scaled=False),
                   model.estimator_coeffs(),
                   MSE_test=MSE_test,
                   p_MSE = p_MSE,
                   p_reg = p_reg,
                   tau = tau_,
                   alpha=alpha_,
                  lambda_=lambda_,
                  mn=mn)

            # ================== Sparsity update =============
            # Updating sparsity 
            update_sparsity = sparsity_scheduler(iteration, torch.sum(MSE_test), model, optimizer)
            if update_sparsity: 
                model.constraint.sparsity_masks = model.sparse_estimator(thetas, time_derivs)

            # ================= Checking convergence
            l1_norm = torch.sum(torch.abs(torch.cat(model.constraint_coeffs(sparse=True, scaled=True), dim=1)))
            converged = convergence(iteration, l1_norm)
            if converged:
                break
    logger.close(model)
    

In [46]:
def train_SBL(model: DeepMoD,
          data: torch.Tensor,
          target: torch.Tensor,
          optimizer,
          extra_params, 
          sparsity_scheduler,
          split = 0.8,
          exp_ID: str = None,
          log_dir: str = None,
          max_iterations: int = 10000,
          write_iterations: int = 25,
          **convergence_kwargs) -> None:
    """Trains the DeepMoD model. This function automatically splits the data set in a train and test set. 

    Args:
        model (DeepMoD):  A DeepMoD object.
        data (torch.Tensor):  Tensor of shape (n_samples x (n_spatial + 1)) containing the coordinates, first column should be the time coordinate.
        target (torch.Tensor): Tensor of shape (n_samples x n_features) containing the target data.
        optimizer ([type]):  Pytorch optimizer.
        sparsity_scheduler ([type]):  Decides when to update the sparsity mask.
        split (float, optional):  Fraction of the train set, by default 0.8.
        exp_ID (str, optional): Unique ID to identify tensorboard file. Not used if log_dir is given, see pytorch documentation.
        log_dir (str, optional): Directory where tensorboard file is written, by default None.
        max_iterations (int, optional): [description]. Max number of epochs , by default 10000.
        write_iterations (int, optional): [description]. Sets how often data is written to tensorboard and checks train loss , by default 25.
    """
    logger = Logger(exp_ID, log_dir)
    sparsity_scheduler.path = logger.log_dir # write checkpoint to same folder as tb output.
    
    t, b, a = extra_params
    
    # Splitting data, assumes data is already randomized
    n_train = int(split * data.shape[0])
    n_test = data.shape[0] - n_train
    data_train, data_test = torch.split(data, [n_train, n_test], dim=0)
    target_train, target_test = torch.split(target, [n_train, n_test], dim=0)
    

    threshold = 1e4
    n_samples = data_train.shape[0]
   
    
    # Training
    convergence = Convergence(**convergence_kwargs)
    for iteration in torch.arange(0, max_iterations):
        # ================== Training Model ============================
        prediction, time_derivs, thetas = model(data_train)
        
        tau_ = torch.min(torch.exp(t), torch.tensor(1e8, dtype=torch.float32))
        alpha_ = torch.min(torch.exp(a), torch.tensor(2e4, dtype=torch.float32)) # we train the log of these things since they're very big
        beta_ = torch.min(torch.exp(b), torch.tensor(1e8, dtype=torch.float32)) # we cap alpha and beta to prevent overflow
                            
        y = time_derivs[0]
        X = thetas[0]
        
        mask = alpha_ < threshold
        X_keep = X[:, mask]
        A_inv = torch.inverse(torch.diag(alpha_[mask]) + beta_ * X_keep.T @ X_keep)
        mn = torch.zeros((a.shape[0], 1)).to(data.device)
        mn[mask, :] = beta_ * A_inv @ X_keep.T @ y
        E = beta_ * torch.sum((y - X @ mn)**2) + (alpha_[:, None] * mn**2).sum()

        p_reg = E - (torch.logdet(A_inv) + n_samples * torch.log(beta_) + torch.sum(torch.log(alpha_))) # we use alpha and lambda since these are bounded
        p_MSE = tau_ * torch.sum((prediction - target_train)**2, dim=0) - n_samples * torch.log(tau_)
        loss = torch.sum(p_MSE + p_reg)
            
        MSE = torch.mean((prediction - target_train)**2, dim=0)  # loss per output
        Reg = torch.stack([torch.mean((dt - theta @ coeff_vector)**2)
                           for dt, theta, coeff_vector in zip(time_derivs, thetas, model.constraint_coeffs(scaled=False, sparse=True))])
       

        # Optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if iteration % write_iterations == 0:
            # ================== Validation costs ================
            with torch.no_grad():
                prediction_test = model.func_approx(data_test)[0]
                MSE_test = torch.mean((prediction_test - target_test)**2, dim=0)  # loss per output
         
            # ====================== Logging =======================
            _ = model.sparse_estimator(thetas, time_derivs) # calculating estimator coeffs but not setting mask
            logger(iteration, 
                   loss, MSE, Reg,
                   model.constraint_coeffs(sparse=True, scaled=True), 
                   model.constraint_coeffs(sparse=True, scaled=False),
                   model.estimator_coeffs(),
                   MSE_test=MSE_test,
                   p_MSE = p_MSE,
                   p_reg = p_reg,
                   tau = tau_,
                   alpha=alpha_,
                   beta_=beta_,
                   mn=mn)

            # ================== Sparsity update =============
            # Updating sparsity 
            update_sparsity = sparsity_scheduler(iteration, torch.sum(MSE_test), model, optimizer)
            #if update_sparsity: 
            #    model.constraint.sparsity_masks = model.sparse_estimator(thetas, time_derivs)

            # ================= Checking convergence
            l1_norm = torch.sum(torch.abs(torch.cat(model.constraint_coeffs(sparse=True, scaled=True), dim=1)))
            converged = convergence(iteration, l1_norm)
            if converged:
                break
    logger.close(model)
    

# Running

In [3]:
# Making dataset
v = 0.1
A = 1.0

x = np.linspace(-3, 4, 100)
t = np.linspace(0.5, 5.0, 50)
x_grid, t_grid = np.meshgrid(x, t, indexing='ij')
dataset = Dataset(BurgersDelta, v=v, A=A)
X, y = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=1000, noise=0.8, random=True, normalize=False)
X, y = X.to(device), y.to(device)
    

In [5]:
t = torch.nn.Parameter(-torch.log(torch.var(y)))
a = torch.nn.Parameter(-torch.log(torch.var(y)))
l  = torch.nn.Parameter(torch.zeros(1))

In [6]:
network = NN(2, [30, 30, 30, 30, 30], 1)
library = Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.0) # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model

sparsity_scheduler = Periodic(periodicity=50, initial_iteration=1e8)

In [7]:
optimizer = torch.optim.Adam([{'params':model.parameters(), 'betas':(0.99, 0.999), 'amsgrad':True, 'lr':2e-3}, {'params':[t, a, l], 'betas':(0.99, 0.999), 'amsgrad':True, 'lr':2e-3}]) # Defining optimizer

In [8]:
train_bayes(model, X, y, optimizer, [t, a, l], sparsity_scheduler, exp_ID='bayes_regression', split=0.8, write_iterations=50, max_iterations=20000, delta=0.0, patience=200) 

 14150  MSE: 2.12e-02  Reg: 1.64e-07  L1: 1.94e+00 

KeyboardInterrupt: 

# SBL

In [47]:
# Making dataset
v = 0.1
A = 1.0

x = np.linspace(-3, 4, 100)
t = np.linspace(0.5, 5.0, 50)
x_grid, t_grid = np.meshgrid(x, t, indexing='ij')
dataset = Dataset(BurgersDelta, v=v, A=A)
X, y = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=1000, noise=0.8, random=True, normalize=False)
X, y = X.to(device), y.to(device)
    

In [48]:
t = torch.nn.Parameter(-torch.log(torch.var(y)))
b = torch.nn.Parameter(-torch.log(torch.var(y)))
a  = torch.nn.Parameter(torch.zeros(12))

In [49]:
network = NN(2, [30, 30, 30, 30, 30], 1)
library = Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.1) # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model
sparsity_scheduler = Periodic(initial_iteration=10**8)

In [51]:
optimizer = torch.optim.Adam([{'params':model.parameters(), 'amsgrad':True, 'lr':2e-3, 'betas':(0.99, 0.999)}, {'params':[t, b, a], 'amsgrad':True, 'lr':2e-3, 'betas':(0.99, 0.999)}]) # Defining optimizer

In [52]:
train_SBL(model, X, y, optimizer, [t, b, a], sparsity_scheduler, exp_ID='SBL_betas', split=0.8, write_iterations=50, max_iterations=30000, delta=0.0, patience=200) 

 29950  MSE: 2.15e-02  Reg: 7.18e-11  L1: 1.83e+00 Algorithm converged. Writing model to disk.


In [20]:
l

Parameter containing:
tensor([5.2740, 3.1534, 5.1648, 5.3445, 2.8648, 2.6787, 4.5340, 5.3478, 0.4639,
        4.4162, 4.8287, 5.0739], requires_grad=True)

In [6]:
from deepymod.data import Dataset
from deepymod.data.kdv import DoubleSoliton


x = np.linspace(-4, 3, 1000)
t = np.linspace(0.0, 2.0, 40)

x_grid, t_grid = np.meshgrid(x, t, indexing='ij')
dataset = Dataset(DoubleSoliton, c=[5.0, 1.0], x0=[-5.0, -1.0])
X, y = dataset.create_dataset(x_grid.reshape(-1, 1), t_grid.reshape(-1, 1), n_samples=1000, noise=0.2, normalize=False, random=True)

In [7]:
t = torch.nn.Parameter(-torch.log(torch.var(y)))
a = torch.nn.Parameter(-torch.log(torch.var(y)))
l  = torch.nn.Parameter(torch.zeros(12))

In [8]:
network = NN(2, [30, 30, 30, 30, 30], 1)
library = Library1D(poly_order=2, diff_order=3) # Library function
estimator = Threshold(0.0) # Sparse estimator 
constraint = LeastSquares() # How to constrain
model = DeepMoD(network, library, estimator, constraint).to(device) # Putting it all in the model

sparsity_scheduler = Periodic(periodicity=50, initial_iteration=1e8)

In [9]:
optimizer = torch.optim.Adam([{'params':model.parameters(), 'betas':(0.99, 0.999), 'amsgrad':True, 'lr':5e-4}, {'params':[t, a, l], 'betas':(0.99, 0.999), 'amsgrad':True, 'lr':5e-4}]) # Defining optimizer

In [10]:
train_SBL(model, X, y, optimizer, [t, a, l], sparsity_scheduler, exp_ID='kdv_slow_4', split=0.8, write_iterations=50, max_iterations=20000, delta=0.0, patience=200) 

 19950  MSE: 2.64e-02  Reg: 7.76e-03  L1: 3.06e+00 Algorithm converged. Writing model to disk.
